# VacationPy
----
#### By: Tameka Kuar
#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create a data frame from cities.csv file
cities_df = pd.read_csv('../output_data/cities.csv')

# Display data frame
cities_df.head(15)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,meulaboh,4.14,96.13,77.27,87,80,4.23,ID,1593541682
1,severo-kurilsk,50.68,156.12,46.02,87,81,16.87,RU,1593541682
2,berlevag,70.86,29.09,41.00,100,75,24.16,NO,1593541683
3,aguas formosas,-17.08,-40.94,68.88,90,100,3.74,BR,1593541683
4,ushuaia,-54.80,-68.30,32.00,92,1,1.95,AR,1593541589
5,chlorakas,34.80,32.41,80.01,88,0,8.05,CY,1593541684
6,jamestown,42.10,-79.24,78.01,60,75,10.29,US,1593541684
7,port lincoln,-34.73,135.87,54.34,81,0,16.67,AU,1593541684
8,mataura,-46.19,168.86,39.00,83,68,3.00,NZ,1593541685
9,vaini,-21.20,-175.20,71.60,94,75,8.05,TO,1593541685


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store cities' humidity in humidity
humidity = cities_df["Humidity"]

# Get the max humidity
maxhumidity = humidity.max()

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                 max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * A max temperature lower than 80 degrees but higher than 70.
    * Humidity is higher than 20% but lower than 70%
    * Wind speed less than 10 mph.
    * Less than 10% cloudiness.


Drop any rows that don't contain all the conditions. You want to be sure the weather is ideal.
* Drop any rows will null values.

In [5]:
# Create a data frame with cities that fit the above conditions
perfect_citites_df = cities_df.loc[(cities_df['Max Temp'] > 70) & 
                                   (cities_df['Max Temp'] < 80) &
                                   (cities_df['Humidity'] > 20) &
                                   (cities_df['Humidity'] < 70) & 
                                   (cities_df['Wind Speed'] < 10) &
                                   (cities_df['Cloudiness'] < 10)]

# Drop rows that don't meet the above conditions
perfect_citites_df = perfect_citites_df.dropna(how='any').reset_index().drop(columns='index')

# Display data frame
perfect_citites_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hami,42.80,93.45,75.00,23,0,7.11,CN,1593541733
1,netivot,31.42,34.59,79.00,69,0,4.41,IL,1593541752
2,tsagan aman,47.56,46.72,72.64,30,0,7.96,RU,1593541777
3,havre-saint-pierre,50.23,-63.60,71.60,64,1,8.05,CA,1593541783
4,kobylka,52.34,21.20,73.99,50,1,1.01,PL,1593541837


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel data frame
hotel_df = perfect_citites_df

# List to hold the hotel names
hotels = []
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# Use iterrows to iterate through the hotel data frame
for index, row in hotel_df.iterrows():
    
    # Set coordinates     
    lat = row['Lat']
    lng = row['Lng']
    
    # Find the hotels within 5000 meters of the coordinates
    params = {
        'location': f'{lat},{lng}',
        'radius': 5000,
        'types': 'hotel',
        'key': g_key
    }
    
    # Assemble url and make API request
    response = requests.get(url, params=params).json()
    
    # Extract results
    results = response['results']
    
    try:
        hotels.append(results[1]['name'])
    except:
        pass


In [7]:
# Add Hotel Name column to hotel data frame
hotel_df["Hotel Name"] = hotels

# Display data frame
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,hami,42.80,93.45,75.00,23,0,7.11,CN,1593541733,Kumul Mansion Scenic Area
1,netivot,31.42,34.59,79.00,69,0,4.41,IL,1593541752,"גן הפקאן א.ד אירועים ושרותי קייטרינג בע""מ"
2,tsagan aman,47.56,46.72,72.64,30,0,7.96,RU,1593541777,Russian Agricultural Bank
3,havre-saint-pierre,50.23,-63.60,71.60,64,1,8.05,CA,1593541783,Auberge Niapiskau
4,kobylka,52.34,21.20,73.99,50,1,1.01,PL,1593541837,Hotel Trylogia


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))